In [1]:
%load_ext autoreload
%autoreload 2

import subprocess
from google.cloud import storage
import os
import PIL
import terra
import pandas as pd
import numpy as np
import meerkat as mk
import torchxrayvision as xrv
from domino.data.mimic import build_dp, mimic_transform

from domino.evaluate.linear import induce_correlation, check_corr_induction

In [2]:
dp = build_dp(
    dataset_dir="/home/common/datasets/mimic", 
    gcp_project="hai-gcp-fine-grained"
)

task: build_dp, run_id=1267


In [2]:
dp = build_dp.out(1267, load=True)

In [4]:
dp.columns

['dicom_id',
 'subject_id_x',
 'study_id',
 'path',
 'subject_id_1',
 'study_id_1',
 'path_1',
 'subject_id_2',
 'study_id_2',
 'Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Enlarged_Cardiomediastinum',
 'Fracture',
 'Lung_Lesion',
 'Lung_Opacity',
 'No_Finding',
 'Pleural_Effusion',
 'Pleural_Other',
 'Pneumonia',
 'Pneumothorax',
 'Support_Devices',
 'subject_id_3',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'dicom',
 'StudyDate_x',
 'ImageType',
 'TableType',
 'DistanceSourceToDetector',
 'DistanceSourceToPatient',
 'Exposure',
 'ExposureTime',
 'XRayTubeCurrent',
 'FieldOfViewRotation',
 'FieldOfViewOrigin',
 'FieldOfViewHorizontalFlip',
 'ViewPosition',
 'PatientOrientation',
 'BurnedInAnnotation',
 'RequestingService',
 'DetectorPrimaryAngle',
 'DetectorElementPhysicalSize',
 'subject_id_y',
 'StudyDate_y',
 'hadm_id',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insuranc

In [5]:
dp

,dicom_id (NumpyArrayColumn),subject_id_x (NumpyArrayColumn),study_id (NumpyArrayColumn),path (NumpyArrayColumn),subject_id_1 (NumpyArrayColumn),study_id_1 (NumpyArrayColumn),path_1 (NumpyArrayColumn),subject_id_2 (NumpyArrayColumn),study_id_2 (NumpyArrayColumn),Atelectasis (NumpyArrayColumn),...,Pneumonia_uzeros (NumpyArrayColumn),Pneumothorax_uzeros (NumpyArrayColumn),Support_Devices_uzeros (NumpyArrayColumn),patient_orientation_rf (NumpyArrayColumn),young (NumpyArrayColumn),ethnicity_black (NumpyArrayColumn),ethnicity_hisp (NumpyArrayColumn),ethnicity_asian (NumpyArrayColumn),burned_in_annotation (NumpyArrayColumn),gender_male (NumpyArrayColumn)
0,9b314ad7-fbcb0422-6db62dfc-732858d0-a5527d8b,10000935,51178377,files/p10/p10000935/s51178377/9b314ad7-fbcb042...,10000935,51178377,files/p10/p10000935/s51178377.txt,10000935.0,51178377.0,NaN,...,0,0,0,0,0,1,0,0,1,0
1,3b8b1b7d-054490d5-385641e7-ff43d2c8-9505f058,10001176,54684191,files/p10/p10001176/s54684191/3b8b1b7d-054490d...,10001176,54684191,files/p10/p10001176/s54684191.txt,10001176.0,54684191.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,ed9c0dfc-ea25b576-0f8cc069-df4cdf14-0cd60eb7,10001176,54684191,files/p10/p10001176/s54684191/ed9c0dfc-ea25b57...,10001176,54684191,files/p10/p10001176/s54684191.txt,10001176.0,54684191.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,d9db838d-4612fd1e-e45b40a9-3ea30033-26efd8e4,10001401,55350604,files/p10/p10001401/s55350604/d9db838d-4612fd1...,10001401,55350604,files/p10/p10001401/s55350604.txt,10001401.0,55350604.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,52d6c812-26dfed36-97e26b11-1fa88177-e8b8b150,10001884,53880659,files/p10/p10001884/s53880659/52d6c812-26dfed3...,10001884,53880659,files/p10/p10001884/s53880659.txt,10001884.0,53880659.0,NaN,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243329,c09014f4-a597350e-18af189b-348221e2-316c74ef,19983508,57052056,files/p19/p19983508/s57052056/c09014f4-a597350...,19983508,57052056,files/p19/p19983508/s57052056.txt,19983508.0,57052056.0,NaN,...,0,0,0,0,0,0,0,0,0,1
243330,ac0b3fbd-40e0bf99-8c7bda63-98b60f34-2f8d84bf,19984491,55712435,files/p19/p19984491/s55712435/ac0b3fbd-40e0bf9...,19984491,55712435,files/p19/p19984491/s55712435.txt,19984491.0,55712435.0,1.0,...,0,0,0,0,0,0,0,0,0,0
243331,4011e4c8-32695a5c-3bf7cddd-f1602228-d0d391b5,19991743,54080659,files/p19/p19991743/s54080659/4011e4c8-32695a5...,19991743,54080659,files/p19/p19991743/s54080659.txt,19991743.0,54080659.0,NaN,...,0,0,0,0,0,0,0,0,0,0
243332,11f33f3e-7fdf7618-ebcbb376-cff190da-b2ff9081,19992875,50634814,files/p19/p19992875/s50634814/11f33f3e-7fdf761...,19992875,50634814,files/p19/p19992875/s50634814.txt,19992875.0,50634814.0,-1.0,...,0,0,0,0,1,0,0,0,0,1


In [4]:
check_corr_induction(
    dp=build_dp.out(1267),
    attributes=[
        "patient_orientation_rf",
        "young",
        "ethnicity_black",
        "ethnicity_hisp",
        "ethnicity_asian",
        "burned_in_annotation",
        "gender_male",
        "Edema_uzeros",
        "Pleural_Effusion_uzeros",
        "Support_Devices_uzeros",
        "Enlarged_Cardiomediastinum_uzeros",
        "Lung_Lesion_uzeros"
    ],
    corr_start=0,
    corr_end=0.8,
    n=30000
)

task: check_corr_induction, run_id=1270


  0%|          | 0/12 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/home/sabri/code/terra/terra/__init__.py", line 230, in _run
    out = self.fn(**args_dict)
  File "/home/sabri/code/domino/domino/evaluate/linear.py", line 152, in check_corr_induction
    induce_correlation(
  File "/home/sabri/code/domino/domino/evaluate/linear.py", line 74, in induce_correlation
    elif int(n_b1) > df[attr_b].sum():
ValueError: cannot convert float NaN to integer



ValueError: cannot convert float NaN to integer

In [79]:
out = induce_correlation(
    df=dp,
    corr=0,
    attr_a="Pleural_Effusion_uzeros",
    attr_b="young",
    n=30000,
    match_mu=True
)

array(["['A', 'F']", "['F', 'A']", "['F', 'L']", "['F', 'R']",
       "['H', 'L']", "['L', 'F']", "['LP', 'F']", "['R', 'F']"],
      dtype=object)

In [46]:
dp["patient_orientation_rf"]  = dp["PatientOrientation"].data == "['R', 'F']"

In [72]:
dp["young"] = (dp["anchor_age"] < 40)

In [97]:
pd.Series(dp["BurnedInAnnotation"].data).unique()

array(['YES', 'NO'], dtype=object)

In [101]:
dp["BurnedInAnnotation"].mean()

0.0

In [100]:
dp.columns

['dicom_id',
 'subject_id_x',
 'study_id',
 'path',
 'subject_id_1',
 'study_id_1',
 'path_1',
 'subject_id_2',
 'study_id_2',
 'Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Enlarged_Cardiomediastinum',
 'Fracture',
 'Lung_Lesion',
 'Lung_Opacity',
 'No_Finding',
 'Pleural_Effusion',
 'Pleural_Other',
 'Pneumonia',
 'Pneumothorax',
 'Support_Devices',
 'subject_id_3',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'dicom',
 'StudyDate_x',
 'ImageType',
 'TableType',
 'DistanceSourceToDetector',
 'DistanceSourceToPatient',
 'Exposure',
 'ExposureTime',
 'XRayTubeCurrent',
 'FieldOfViewRotation',
 'FieldOfViewOrigin',
 'FieldOfViewHorizontalFlip',
 'ViewPosition',
 'PatientOrientation',
 'BurnedInAnnotation',
 'RequestingService',
 'DetectorPrimaryAngle',
 'DetectorElementPhysicalSize',
 'subject_id_y',
 'StudyDate_y',
 'hadm_id',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insuranc